# Baseline Models

The following Notebook contains the first round of experimentation for the USM Datathon. We show our resulting for 5-Fold Cross Validation metrics (used to determine the generalization power of the model in the competition leaderboard) and the final Test Score measured with the whole Training Data. 

The evaluation Score used during the competition was a Custom Implementation of the F1-Score defined as:

$$F_{1\,custom} = 0.5 \cdot (F_{1\,Hate} + Macro \, F_{1\,communities})$$

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from utils import import_data, validation_train, full_train
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from loguru import logger
import pandas as pd
import numpy as np
import time


### Validation Schema

Training Process following a 5-Fold Cross Validation Schema.

In [3]:
RANDOM_STATE = 42
df_train, df_test, stopwords = import_data()

LABELS = [
    "Odio",
    "Mujeres",
    "Comunidad LGBTQ+",
    "Comunidades Migrantes",
    "Pueblos Originarios",
]

nb = MultinomialNB()
lr = LogisticRegression(random_state=RANDOM_STATE, max_iter=1000)
et = ExtraTreesClassifier(n_estimators=500, random_state=RANDOM_STATE, n_jobs=-1)
rf = RandomForestClassifier(n_estimators=500, random_state=RANDOM_STATE, n_jobs=-1)
cb = CatBoostClassifier(
    n_estimators=500, random_state=RANDOM_STATE, verbose=False, thread_count=-1
)
lgb = LGBMClassifier(n_estimators=500, random_state=RANDOM_STATE, n_jobs=-1)
xgb = XGBClassifier(n_estimators=500, random_state=RANDOM_STATE, n_jobs=-1)
mlp = MLPClassifier(
    hidden_layer_sizes=(64, 32),
    activation="relu",
    solver="adam",
    random_state=RANDOM_STATE,
    alpha=0.1,
)
models_dict = {
    "Naive Bayes": nb,
    "Logistic Regression": lr,
    "Extra Trees": et,
    "Random Forest": rf,
    "Catboost": cb,
    "LightGBM": lgb,
    "XGBoost": xgb,
    "MLP": mlp,
}


2023-09-15 13:16:28.289 | INFO     | utils.utilities:import_data:12 - Training and Test data succesfully loaded...
2023-09-15 13:16:28.290 | INFO     | utils.utilities:import_data:13 - Train Shape: (2256, 9), Test Shape: (2291, 9)


In [4]:
validation_results = []
for name, model in models_dict.items():
    start_time = time.time()
    dict_results = {}
    dict_results["model_name"] = name
    dict_results.update(
        validation_train(df_train, model, LABELS, stopwords, random_state=RANDOM_STATE)
    )
    dict_results["cv_training_time (min)"] = np.round(
        (time.time() - start_time) / 60, 2
    )
    logger.info(f"{name} Results:")
    logger.info(f"Mean Training Score: {dict_results['mean_train_score']}")
    logger.info(f"Mean Test Score: {dict_results['mean_val_score']}")
    validation_results.append(dict_results)

validation_df = pd.DataFrame(validation_results)
validation_df


2023-09-15 13:16:28.656 | INFO     | __main__:<module>:12 - Naive Bayes Results:
2023-09-15 13:16:28.657 | INFO     | __main__:<module>:13 - Mean Training Score: 0.969342010645185
2023-09-15 13:16:28.657 | INFO     | __main__:<module>:14 - Mean Test Score: 0.7284914745443303
2023-09-15 13:16:30.666 | INFO     | __main__:<module>:12 - Logistic Regression Results:
2023-09-15 13:16:30.667 | INFO     | __main__:<module>:13 - Mean Training Score: 0.9767475099969705
2023-09-15 13:16:30.667 | INFO     | __main__:<module>:14 - Mean Test Score: 0.7302634181239027
2023-09-15 13:16:44.988 | INFO     | __main__:<module>:12 - Extra Trees Results:
2023-09-15 13:16:44.988 | INFO     | __main__:<module>:13 - Mean Training Score: 0.9997901410945861
2023-09-15 13:16:44.989 | INFO     | __main__:<module>:14 - Mean Test Score: 0.777196959827281
2023-09-15 13:16:56.820 | INFO     | __main__:<module>:12 - Random Forest Results:
2023-09-15 13:16:56.820 | INFO     | __main__:<module>:13 - Mean Training Score:

,model_name,mean_train_score,sd_train_score,mean_val_score,sd_val_score,mean_precision_train_score,sd_precision_train_score,mean_precision_val_score,sd_precision_val_score,mean_recall_train_score,sd_recall_train_score,mean_recall_val_score,sd_recall_val_score,cv_training_time (min)
0,Naive Bayes,0.969342,0.001314,0.728491,0.023792,0.979976,0.001648,0.830191,0.024515,0.949345,0.002773,0.619482,0.032544,0.01
1,Logistic Regression,0.976748,0.001081,0.730263,0.012259,0.995945,0.000760,0.886027,0.008727,0.945608,0.003457,0.603291,0.020781,0.03
2,Extra Trees,0.999790,0.000171,0.777197,0.025783,1.000000,0.000000,0.876066,0.009494,0.999672,0.000268,0.710161,0.038386,0.24
3,Random Forest,0.999790,0.000171,0.747509,0.015967,0.999962,0.000076,0.875421,0.016485,0.999710,0.000247,0.650014,0.024626,0.20
4,Catboost,0.842930,0.003843,0.767565,0.012398,0.928142,0.002028,0.885169,0.012828,0.763974,0.006187,0.685882,0.015977,1.12
5,LightGBM,0.770482,0.005972,0.676694,0.010509,0.901760,0.008869,0.770532,0.047771,0.664634,0.010542,0.561061,0.011954,0.06
6,XGBoost,0.891976,0.003370,0.752562,0.015310,0.955648,0.002724,0.847096,0.012692,0.827290,0.009450,0.682370,0.021205,0.11
7,MLP,0.999634,0.000208,0.751827,0.017548,1.000000,0.000000,0.863398,0.019239,0.999403,0.000373,0.657323,0.025072,1.12


### Full Train and Predictions

Test Predictions using the Whole Training Set.

In [5]:
training_results = []
for name, model in models_dict.items():
    start_time = time.time()
    dict_results = {}
    dict_results["model_name"] = name
    dict_results.update(full_train(df_train, df_test, model, LABELS, stopwords))
    dict_results["full_training_time (min)"] = np.round(
        (time.time() - start_time) / 60, 2
    )
    logger.info(f"{name} Results:")
    logger.info(f"Test Score: {dict_results['test_score']}")
    training_results.append(dict_results)

training_df = pd.DataFrame(training_results)
training_df


2023-09-15 13:19:22.006 | INFO     | __main__:<module>:10 - Naive Bayes Results:
2023-09-15 13:19:22.007 | INFO     | __main__:<module>:11 - Test Score: 0.7455443267553221
2023-09-15 13:19:22.600 | INFO     | __main__:<module>:10 - Logistic Regression Results:
2023-09-15 13:19:22.600 | INFO     | __main__:<module>:11 - Test Score: 0.7669777805756477
2023-09-15 13:19:25.538 | INFO     | __main__:<module>:10 - Extra Trees Results:
2023-09-15 13:19:25.539 | INFO     | __main__:<module>:11 - Test Score: 0.8032498173030194
2023-09-15 13:19:27.920 | INFO     | __main__:<module>:10 - Random Forest Results:
2023-09-15 13:19:27.920 | INFO     | __main__:<module>:11 - Test Score: 0.7990736226153257
2023-09-15 13:19:42.399 | INFO     | __main__:<module>:10 - Catboost Results:
2023-09-15 13:19:42.399 | INFO     | __main__:<module>:11 - Test Score: 0.7865336482857408
2023-09-15 13:19:43.315 | INFO     | __main__:<module>:10 - LightGBM Results:
2023-09-15 13:19:43.315 | INFO     | __main__:<module>:

,model_name,test_score,test_precision,test_recall,full_training_time (min)
0,Naive Bayes,0.745544,0.751765,0.731662,0.00
1,Logistic Regression,0.766978,0.797755,0.731777,0.01
2,Extra Trees,0.803250,0.768195,0.848739,0.05
3,Random Forest,0.799074,0.789340,0.810810,0.04
4,Catboost,0.786534,0.784202,0.802355,0.24
5,LightGBM,0.699077,0.711117,0.678284,0.02
6,XGBoost,0.782395,0.759743,0.809113,0.02
7,MLP,0.768206,0.758680,0.805170,0.27
